In [15]:
import skimage as sk
from skimage.io import imread, imshow, imsave
import numpy as np
import os
from glob import glob
import napari
import pyclesperanto as cle
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
help(cle.scale)

In [ ]:
def segment_on_GPU(input_image,spot_sigma,outline_sigma,voxel_size_z,voxel_size_y,voxel_size_x):
    input_gpu = cle.push(input_image)
    resampled = cle.create([int(input_gpu.shape[0] * voxel_size_z), int(input_gpu.shape[1] * voxel_size_y), int(input_gpu.shape[2] * voxel_size_x)])
    cle.scale(input_gpu, resampled, factor_x=voxel_size_x, factor_y=voxel_size_y, factor_z=voxel_size_z, centered=False)
    #normalize the image stack
    equalized_intensities_stack = cle.create_like(resampled)
    a_slice = cle.create([resampled.shape[1], resampled.shape[2]])
    num_slices = resampled.shape[0]
    mean_intensity_stack = cle.mean_of_all_pixels(resampled)
    corrected_slice = None
    for z in range(0, num_slices):
        # get a single slice out of the stack
        cle.copy_slice(resampled, a_slice, z)
        # measure its intensity
        mean_intensity_slice = cle.mean_of_all_pixels(a_slice)
        # correct the intensity
        correction_factor = mean_intensity_slice/mean_intensity_stack
        corrected_slice = cle.multiply_image_and_scalar(a_slice, corrected_slice, correction_factor)
        # copy slice back in a stack
        cle.copy_slice(corrected_slice, equalized_intensities_stack, z)
    #background subtraction
    background_subtracted = cle.top_hat_box(equalized_intensities_stack, radius_x=10, radius_y=10, radius_z=10)
    #get seed points
    blurred = cle.gaussian_blur(background_subtracted, sigma_x=spot_sigma, sigma_y=spot_sigma, sigma_z=spot_sigma)
    detected_spots = cle.detect_maxima_box(blurred, radius_x=1, radius_y=1, radius_z=1)
    #segment signal
    blurred = cle.gaussian_blur(background_subtracted, sigma_x=outline_sigma, sigma_y=outline_sigma, sigma_z=outline_sigma)
    copy_blurred = cle.copy(blurred)
    blurred_cpu = cle.pull(copy_blurred)
    thresh = sk.filters.threshold_multiotsu(blurred_cpu, classes=5)
    mask = blurred > thresh[1]
    selected_spots = cle.binary_and(mask, detected_spots)
    voronoi_diagram = cle.masked_voronoi_labeling(selected_spots, mask)
    df = pd.DataFrame(cle.statistics_of_labelled_pixels(resampled, voronoi_diagram))
    #pull data off gpu
    input_pull = cle.pull(input_gpu)
    background_subtracted_pull = cle.pull(background_subtracted)
    equalized_intensities_stack_pull = cle.pull(equalized_intensities_stack)
    mean_intensity_stack_pull = cle.pull(mean_intensity_stack)
    blurred_pull = cle.pull(blurred)
    voronoi_diagram_pull = cle.pull(voronoi_diagram)
    resampled_pull = cle.pull(resampled)
    detected_spots_pull = cle.pull(detected_spots)
    selected_spots_pull = cle.pull(selected_spots)
    mask_pull = cle.pull(mask)
    return voronoi_diagram_pull, resampled_pull, detected_spots_pull, selected_spots_pull, df

In [16]:
def segment_on_GPU_no_resample(input_image,spot_sigma,outline_sigma,voxel_size_z,voxel_size_y,voxel_size_x):
    input_gpu = cle.push(input_image)
    equalized_intensities_stack = cle.create_like(input_gpu)
    a_slice = cle.create([input_gpu.shape[1], input_gpu.shape[2]])
    num_slices = input_gpu.shape[0]
    mean_intensity_stack = cle.mean_of_all_pixels(input_gpu)
    corrected_slice = None
    for z in range(0, num_slices):
        # get a single slice out of the stack
        cle.copy_slice(input_gpu, a_slice, z)
        # measure its intensity
        mean_intensity_slice = cle.mean_of_all_pixels(a_slice)
        # correct the intensity
        correction_factor = mean_intensity_slice/mean_intensity_stack
        corrected_slice = cle.multiply_image_and_scalar(a_slice, corrected_slice, correction_factor)
        # copy slice back in a stack
        cle.copy_slice(corrected_slice, equalized_intensities_stack, z)
    #background subtraction
    background_subtracted_top_hat = cle.top_hat_sphere(equalized_intensities_stack,radius_x=10,radius_y=10,radius_z=10)
    #get seed points
    blurred = cle.gaussian_blur(background_subtracted_top_hat, sigma_x=spot_sigma, sigma_y=spot_sigma, sigma_z=spot_sigma)
    detected_spots = cle.detect_maxima_box(blurred, radius_x=2, radius_y=2, radius_z=2)
    #segment signal
    blurred = cle.gaussian_blur(background_subtracted_top_hat, sigma_x=outline_sigma, sigma_y=outline_sigma, sigma_z=outline_sigma)
    copy_blurred = cle.copy(blurred)
    blurred_cpu = cle.pull(copy_blurred)
    #add in extra filter here?
    thresh = sk.filters.threshold_multiotsu(blurred_cpu, classes=5)
    mask = blurred > thresh[1]
    selected_spots = cle.binary_and(mask, detected_spots)
    voronoi_diagram = cle.masked_voronoi_labeling(selected_spots, mask)
    df = pd.DataFrame(cle.statistics_of_labelled_pixels(input_gpu, voronoi_diagram))
    #pull data off gpu
    input_pull = cle.pull(input_gpu)
    #background_subtracted_pull = cle.pull(background_subtracted)
    background_subtracted_top_hat_pull = cle.pull(background_subtracted_top_hat)
    equalized_intensities_stack_pull = cle.pull(equalized_intensities_stack)
    mean_intensity_stack_pull = cle.pull(mean_intensity_stack)
    blurred_pull = cle.pull(blurred)
    voronoi_diagram_pull = cle.pull(voronoi_diagram)
    #resampled_pull = cle.pull(resampled)
    detected_spots_pull = cle.pull(detected_spots)
    selected_spots_pull = cle.pull(selected_spots)
    mask_pull = cle.pull(mask)
    return voronoi_diagram_pull, detected_spots_pull, selected_spots_pull, df, background_subtracted_top_hat_pull,blurred_pull

In [30]:
def convert_voxels_to_volume(df,voxel):
    #areas = np.asarray(df.['area']].astype(int))
    scaled_vol = []
    for a in np.asarray(df.loc[df['area']]).astype(int):
        scaled_vol.append(a*voxel)
    scaled_df = pd.Series(scaled_vol,name='Volume(um^3)')
    merged_df = pd.concat([df,scaled_df],axis=1)
    return merged_df

SyntaxError: unmatched ')' (3540692544.py, line 4)

In [18]:
def get_3D_meshes(mask_img,scaled_df):
    array = []
    for i in np.asarray(scaled_df['label']).astype(int):
        obj = mask_img == i
        verts, faces, _ , _ = sk.measure.marching_cubes(obj, level=0.0)
        surf_area = sk.measure.mesh_surface_area(verts, faces)
        array.append(surf_area)
    surface_areas = pd.Series(array,name='Surface_Area(um^2)')
    merged_df = pd.concat([scaled_df,surface_areas], axis=1)
    return merged_df

In [19]:
def save(save_path, img_name, mask_img, filtered_mask, detected_spots, selected_spots, merged_df, background_subtracted):
    # make directories if they do not exist
    try:
        masks_path = os.mkdir(os.path.join(save_path,'masks'))
        filtered_masks_path = os.mkdir(os.path.join(save_path,'filtered_masks'))
        detected_spots_path = os.mkdir(os.path.join(save_path,'detected_spots'))
        selected_spots_path = os.mkdir(os.path.join(save_path,'selected_spots'))
        dataframe_path = os.mkdir(os.path.join(save_path,'measurements'))
        background_subtracted_path = os.mkdir(os.path.join(save_path,'background_subtracted_imgs'))
    # use the expected paths if the directories exist already
    except:
        masks_path = os.path.join(save_path,'masks')
        filtered_masks_path = os.path.join(save_path,'filtered_masks')
        detected_spots_path = os.path.join(save_path,'detected_spots')
        selected_spots_path = os.path.join(save_path,'selected_spots')
        dataframe_path = os.path.join(save_path,'measurements')
        background_subtracted_path = os.path.join(save_path,'background_subtracted_imgs')
    imsave(os.path.join(masks_path,'masks_'+img_name+'.tif'),mask_img)
    imsave(os.path.join(filtered_masks_path,'filtered_masks_'+img_name+'.tif'),filtered_mask)
    imsave(os.path.join(detected_spots_path,'detected_spots_'+img_name+'.tif'),detected_spots)
    imsave(os.path.join(selected_spots_path,'selected_spots_'+img_name+'.tif'),selected_spots)
    merged_df.to_csv(os.path.join(dataframe_path,'measurements_'+img_name+'.csv'))
    imsave(os.path.join(background_subtracted_path,'background_subtracted_'+img_name+'.tif'),background_subtracted)

In [3]:
cle.select_device("NVIDIA")

(OpenCL) NVIDIA RTX A4000 (OpenCL 3.0 CUDA)
	Vendor:                      NVIDIA Corporation
	Driver Version:              572.83
	Device Type:                 GPU
	Compute Units:               48
	Global Memory Size:          16375 MB
	Local Memory Size:           0 MB
	Maximum Buffer Size:         4093 MB
	Max Clock Frequency:         1560 MHz
	Image Support:               Yes

### Reading in images

In [20]:
#set voxel size for images
voxel_size_x = 0.301
voxel_size_y = 0.301
voxel_size_z = 4.55
voxel = 0.301*0.301*4.55

In [21]:
CMO_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/Control_MO/*.tif'))
GMO_1ng_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/GNAS_MO_1ng/*.tif'))
GMO_2ng_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/GNAS_MO_2ng/*.tif'))

In [22]:
CMO_imgs = list(map(sk.io.imread,CMO_files))
GMO_1ng_imgs = list(map(sk.io.imread,GMO_1ng_files))
GMO_2ng_imgs = list(map(sk.io.imread,GMO_2ng_files))

In [8]:
print(len(GMO_1ng_imgs))

8


In [23]:
#testing on individual images
input_image = GMO_2ng_imgs[1]
spot_sigma = 9
outline_sigma = 1
voronoi_diagram, detected_spots, selected_spots, df, background_subtracted_top_hat,blurred = segment_on_GPU_no_resample(
    input_image,spot_sigma,outline_sigma,voxel_size_z,voxel_size_y,voxel_size_x)

In [29]:
scaled_df = convert_voxels_to_volume(df,voxel)

KeyError: '[5450.0, 16334.0, 19001.0, 18587.0, 32346.0, 17683.0, 13934.0, 19217.0, 24557.0, 27057.0, 25006.0, 17705.0, 27758.0, 45134.0, 27954.0, 31478.0, 50042.0, 37220.0, 53952.0, 33862.0, 24183.0, 51204.0, 40409.0, 8848.0, 32062.0, 26778.0, 49841.0, 49754.0, 12596.0, 35251.0, 21134.0, 21697.0, 35676.0, 34370.0, 28704.0, 12854.0, 7119.0, 8222.0, 24651.0, 24639.0, 16911.0, 15687.0, 34794.0, 24772.0, 8852.0, 15795.0, 25472.0, 9232.0, 16394.0, 24178.0, 15709.0, 23885.0, 26705.0, 20214.0, 21602.0, 21000.0, 35296.0, 34532.0, 24712.0, 18813.0, 22896.0, 10629.0, 15324.0, 32425.0, 11313.0, 14710.0, 42016.0, 23195.0, 10312.0, 17395.0, 8757.0, 13482.0, 28205.0, 16134.0, 7238.0, 21893.0, 14320.0, 9204.0, 10740.0, 16398.0, 20315.0, 27691.0, 28141.0, 18317.0, 20249.0, 8136.0, 19392.0, 11987.0, 28975.0, 11772.0, 9239.0, 17999.0, 14767.0, 18676.0, 13169.0, 11361.0, 21238.0, 21614.0, 28841.0, 14655.0, 14384.0, 16895.0, 30010.0, 29124.0, 16843.0, 19880.0, 25109.0, 11696.0, 6814.0, 26405.0, 44845.0, 7

In [14]:
viewer = napari.view_image(background_subtracted_top_hat_pull,name='img',scale=[voxel_size_z,voxel_size_y,voxel_size_x])
viewer.add_image(voronoi_diagram_pull, name='masks',scale=[voxel_size_z,voxel_size_y,voxel_size_x])

<Image layer 'masks' at 0x24e58163460>

In [ ]:
props = sk.measure.regionprops_table(voronoi_diagram_pull,background_subtracted_pull,properties=('label','area','intensity_mean'))
props_df = pd.DataFrame(props)
props_df.tail()
filtered_props = props_df.loc[props_df['intensity_mean']>25]
filtered_props.head()
   

In [ ]:
fig,ax = plt.subplots()
props_df.plot(x='label',y='intensity_mean',kind='scatter',ax=ax)
plt.show()

In [ ]:
filtered_mask = np.isin(voronoi_diagram_pull, np.asarray(filtered_props['label'])).astype(int)

In [ ]:
filtered_labels = filtered_mask*voronoi_diagram_pull

In [ ]:
viewer = napari.view_image(background_subtracted_pull,name='img',scale=[voxel_size_z,voxel_size_y,voxel_size_x])
viewer.add_image(voronoi_diagram_pull, name='masks',scale=[voxel_size_z,voxel_size_y,voxel_size_x])
viewer.add_image(filtered_labels,name='filtered',scale=[voxel_size_z,voxel_size_y,voxel_size_x])

In [ ]:
viewer = napari.view_image(voronoi_diagram_pull, name='masks',scale=[voxel_size_z,voxel_size_y,voxel_size_x])
viewer.add_image(input_image,name='img',scale=[voxel_size_z,voxel_size_y,voxel_size_x])
viewer.add_image(background_subtracted_pull,name='equal',scale=[voxel_size_z,voxel_size_y,voxel_size_x])

## Plan: preprocess, threshold signal, connected components, post-process objects, measure volume and SA to get ratio, measure fluorescence values, PCA on volume/SA to find threshold for splitting into active and inactive state?

#### Trying to get GPU use working

In [ ]:
test_img = CMO_imgs[6]

In [ ]:
input_gpu = cle.push(test_img)
input_gpu

Inspect 3D data

In [ ]:
def show(image_to_show, labels=False):
    """
    This function generates three projections: in X-, Y- and Z-direction and shows them.
    """
    projection_x = cle.maximum_x_projection(image_to_show)
    projection_y = cle.maximum_y_projection(image_to_show)
    projection_z = cle.maximum_z_projection(image_to_show)

    fig, axs = plt.subplots(1, 3, figsize=(15, 15))
    axs[0].imshow(projection_x, cmap='prism' if labels else 'gray')
    axs[1].imshow(projection_y, cmap='prism' if labels else 'gray')
    axs[2].imshow(projection_z, cmap='prism' if labels else 'gray')
    plt.show()

show(input_gpu)
print(input_gpu.shape)

In [ ]:
#Resample to make isotropic
resampled = cle.create([int(input_gpu.shape[0] * voxel_size_z), int(input_gpu.shape[1] * voxel_size_y), int(input_gpu.shape[2] * voxel_size_x)])
cle.scale(input_gpu, resampled, factor_x=voxel_size_x, factor_y=voxel_size_y, factor_z=voxel_size_z, centered=False)

show(resampled)
print(resampled.shape)

May not have enough Z resolution for resonable reasults

In [ ]:
equalized_intensities_stack = cle.create_like(resampled)
a_slice = cle.create([resampled.shape[1], resampled.shape[2]])

num_slices = resampled.shape[0]
mean_intensity_stack = cle.mean_of_all_pixels(resampled)

corrected_slice = None
for z in range(0, num_slices):
    # get a single slice out of the stack
    cle.copy_slice(resampled, a_slice, z)
    # measure its intensity
    mean_intensity_slice = cle.mean_of_all_pixels(a_slice)
    # correct the intensity
    correction_factor = mean_intensity_slice/mean_intensity_stack
    corrected_slice = cle.multiply_image_and_scalar(a_slice, corrected_slice, correction_factor)
    # copy slice back in a stack
    cle.copy_slice(corrected_slice, equalized_intensities_stack, z)

#show(equalized_intensities_stack)

In [ ]:
#Correct background
background_subtracted = cle.top_hat_box(equalized_intensities_stack, radius_x=10, radius_y=10, radius_z=10)
#show(background_subtracted)

In [ ]:
segmented = cle.voronoi_otsu_labeling(background_subtracted, spot_sigma=5, outline_sigma=2)
show(segmented, labels=True)

In [ ]:
help(cle.detect_maxima_box)

In [ ]:
sigma_spot_detection = 7
blurred = cle.gaussian_blur(background_subtracted, sigma_x=sigma_spot_detection, sigma_y=sigma_spot_detection, sigma_z=sigma_spot_detection)
#enough of a blue to remove most of the texture of the objects wihle retaining their presence

detected_spots = cle.detect_maxima_box(blurred, radius_x=1, radius_y=1, radius_z=1)

number_of_spots = cle.sum_of_all_pixels(detected_spots)
print("number of detected spots", number_of_spots)
#viewer = napari.view_image(blurred, name='blurred')
#viewer.add_image(detected_spots, name='spots')
# fig, axs = plt.subplots(1, 2, figsize=(15, 15))
# axs[0].imshow(blurred[95,300:500,300:])
# axs[1].imshow(detected_spots[95,300:500,300:])
# plt.show()

In [ ]:
sigma_outline = 1
blurred = cle.gaussian_blur(background_subtracted, sigma_x=sigma_outline, sigma_y=sigma_outline, sigma_z=sigma_outline)
#Smaller blur to make threhsolding easier and maintain reasonable boundaries

# binary = cle.threshold_otsu(blurred)
# viewer = napari.view_image(blurred, name='blurred')
# viewer.add_image(binary, name='mask')

In [ ]:
copy_blurred = cle.copy(blurred)
copy_background_subtracted = cle.copy(background_subtracted)

In [ ]:
help(cle.greater)

In [ ]:
blurred_cpu = cle.pull(copy_blurred)
iso_norm = cle.pull(copy_background_subtracted)

In [ ]:
#testing thresholds

thresh = sk.filters.threshold_multiotsu(blurred_cpu, classes=5)
print(thresh)
mask = blurred>thresh[1]

# viewer = napari.view_image(blurred, name='blurred')
# viewer.add_image(mask, name='mask')
# viewer.add_image(iso_norm, name='iso data')

In [ ]:
viewer = napari.view_image(iso_norm, name='iso data')
viewer.add_image(blurred, name='blurred')
viewer.add_image(mask, name='mask')

In [ ]:
mask_gpu = blurred > thresh[1]
viewer = napari.view_image(mask_gpu, name='mask')

In [ ]:
selected_spots = cle.binary_and(mask_gpu, detected_spots)

number_of_spots = cle.sum_of_all_pixels(selected_spots)
print("number of selected spots", number_of_spots)

viewer = napari.view_image(mask_gpu, name='mask')
viewer.add_image(detected_spots, name='Original Spots')
viewer.add_image(selected_spots, name='Filtered Spots')

In [ ]:
voronoi_diagram = cle.masked_voronoi_labeling(selected_spots, mask_gpu)
viewer = napari.view_image(voronoi_diagram, name='mask')
viewer.add_image(background_subtracted, name='blurred_img')

In [ ]:
df = pd.DataFrame(
    cle.statistics_of_labelled_pixels(resampled, voronoi_diagram)
    )
df.head()
df.to_csv(os.path.join(save_df_path,'measurements_'+clean_name+'.csv'))

## Creating Functions

In [ ]:
def segment_on_GPU(input_image,spot_sigma,outline_sigma,voxel_size_z,voxel_size_y,voxel_size_x):
    input_gpu = cle.push(input_image)
    resampled = cle.create([int(input_gpu.shape[0] * voxel_size_z), int(input_gpu.shape[1] * voxel_size_y), int(input_gpu.shape[2] * voxel_size_x)])
    cle.scale(input_gpu, resampled, factor_x=voxel_size_x, factor_y=voxel_size_y, factor_z=voxel_size_z, centered=False)
    #normalize the image stack
    equalized_intensities_stack = cle.create_like(resampled)
    a_slice = cle.create([resampled.shape[1], resampled.shape[2]])
    num_slices = resampled.shape[0]
    mean_intensity_stack = cle.mean_of_all_pixels(resampled)
    corrected_slice = None
    for z in range(0, num_slices):
        # get a single slice out of the stack
        cle.copy_slice(resampled, a_slice, z)
        # measure its intensity
        mean_intensity_slice = cle.mean_of_all_pixels(a_slice)
        # correct the intensity
        correction_factor = mean_intensity_slice/mean_intensity_stack
        corrected_slice = cle.multiply_image_and_scalar(a_slice, corrected_slice, correction_factor)
        # copy slice back in a stack
        cle.copy_slice(corrected_slice, equalized_intensities_stack, z)
    #background subtraction
    background_subtracted = cle.top_hat_box(equalized_intensities_stack, radius_x=10, radius_y=10, radius_z=10)
    #get seed points
    blurred = cle.gaussian_blur(background_subtracted, sigma_x=spot_sigma, sigma_y=spot_sigma, sigma_z=spot_sigma)
    detected_spots = cle.detect_maxima_box(blurred, radius_x=1, radius_y=1, radius_z=1)
    #segment signal
    blurred = cle.gaussian_blur(background_subtracted, sigma_x=outline_sigma, sigma_y=outline_sigma, sigma_z=outline_sigma)
    copy_blurred = cle.copy(blurred)
    blurred_cpu = cle.pull(copy_blurred)
    thresh = sk.filters.threshold_multiotsu(blurred_cpu, classes=5)
    mask = blurred > thresh[1]
    selected_spots = cle.binary_and(mask, detected_spots)
    voronoi_diagram = cle.masked_voronoi_labeling(selected_spots, mask)
    df1 = pd.DataFrame(cle.statistics_of_labelled_pixels(resampled, voronoi_diagram))
    voronoi_diagram_cpu = cle.pull(voronoi_diagram)
    resampled_cpu = cle.pull(resampled)
    # meshes = sk.measure.marching_cubes(voronoi_diagram_cpu, spacing=voxels)
    # df2 = sk.measure.regionprops_table(voronoi_diagram_cpu, resampled_cpu,properties=('label','area','intensity_max','intensity_mean','intensity_min'))
    # df2 = pd.DataFrame.from_dict(df2)
    #df1.to_csv(os.path.join('E:/ParadaKusz_Lab/Test_Outputs','test_clemeasurements.csv'))
    # df2.to_csv(os.path.join('E:/ParadaKusz_Lab/Test_Outputs','test_skmeasurements.csv'))
    return voronoi_diagram_cpu, resampled_cpu, df1 
    

In [ ]:
test_img = CMO_imgs[1]
voroni_diagram, resampled_img, df = segment_on_GPU(test_img,7,1,voxels[0],voxels[1],voxels[2])

In [ ]:
df.head()

### Testing a bounding box crop approach to getting surface area measurements (too complicated)

In [ ]:
bbox_min_x = np.asarray(df.loc[:,'bbox_min_x'], dtype=int)
bbox_min_y = np.asarray(df.loc[:,'bbox_min_y'], dtype=int)
bbox_min_z = np.asarray(df.loc[:,'bbox_min_z'], dtype=int)
width = np.asarray(df.loc[:,'bbox_width'], dtype=int)
height = np.asarray(df.loc[:,'bbox_height'], dtype=int)
depth = np.asarray(df.loc[:,'bbox_depth'], dtype=int)

In [ ]:
bbox_min_x

In [ ]:
obj = cle.crop(voroni_diagram, 
               start_x=bbox_min_x[0],start_y=bbox_min_y[0],start_z=bbox_min_z[0],width=width[0],height=height[0],depth=depth[0])
viewer = napari.view_image(obj, name='test')

In [ ]:
obj_cpu = cle.pull(obj)
verts, faces, _ , _ = sk.measure.marching_cubes(obj_cpu, level=0.0,spacing=voxels)
surf_area = sk.measure.mesh_surface_area(verts, faces)

In [ ]:
help(cle.crop)

### Testing a threshold-type approach to getting surface area measurements (easier and less room for error)

In [ ]:
obj_thresh = voroni_diagram == 8
viewer = napari.view_image(obj_thresh, name='test')

In [ ]:
labels_cpu = cle.pull(voroni_diagram)
max_lab = int(np.max(labels_cpu))
max_lab

In [ ]:
def get_3D_meshes(mask_img):
    array = []
    labels_cpu = cle.pull(mask_img)
    for i in range(1,int(np.max(labels_cpu)+1)):
        obj = labels_cpu == int(i)
        verts, faces, _ , _ = sk.measure.marching_cubes(obj, level=0.0)
        surf_area = sk.measure.mesh_surface_area(verts, faces)
        array.append(surf_area)
    measurements = pd.Series(array,name='Surface_Area')
    return measurements

In [ ]:
surface_areas = get_3D_meshes(voroni_diagram)

In [ ]:
surface_areas.tail()

In [ ]:
merged_df = pd.concat([df,surface_areas], axis=1)


In [ ]:
merged_df.to_csv(os.path.join('E:/ParadaKusz_Lab/Test_Outputs','test_mergedmeasurements2.csv'))

In [ ]:
verify = sk.measure.regionprops_table(labels_cpu, labels_cpu, properties=('label','intensity_max'))
verify_df = pd.DataFrame.from_dict(verify)
verify_df.to_csv(os.path.join('E:/ParadaKusz_Lab/Test_Outputs','test_verifylabels.csv'))

In [ ]:
#testing function
voroni_diagram, resampled_img = segment_on_GPU(test_img,7,1,voxels[0],voxels[1],voxels[2])
voroni_diagram.shape

In [ ]:
np.max(voroni_diagram)

In [ ]:
#Creating meshes and measuring volume surface area
obj_cpu = cle.pull(obj_thresh)
verts, faces, _ , _ = sk.measure.marching_cubes(obj_cpu, level=0.0,spacing=voxels)
surf_area = sk.measure.mesh_surface_area(verts, faces)

In [ ]:
print(surf_area)

In [ ]:
def saving():
     #save everything
    equalized_intensities_stack_cpu = cle.pull(equalized_intensities_stack)
    imsave(os.path.join(cp_masks_path,'equalized_resampled_'+files[i]), equalized_intensities_stack_cpu)
    